In [1]:
import numpy as np

In [2]:
def grad_phi(l, w, B):
    
    size = len(w)
    
#     print(np.dot(B, np.ones(size)).A1)
    grad_phi_1 = l - np.dot(B, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    grad_phi_2 = w - np.dot(B.T, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    
    return grad_phi_1, grad_phi_2, np.concatenate((grad_phi_1.A1, grad_phi_2.A1))

In [3]:
np.dot(np.array([1,2,3]), np.array([1, 2, 3]))

14

In [4]:
def phi(x, l, w, B):
    
    size = len(w)
    
    return np.dot(x[:size], l) + np.dot(x[size:], w) + np.log(np.dot(np.dot(np.ones(size).T, B), np.ones(size)).A1[0])

In [5]:
# np.dot(x[:size], l)

In [6]:
def f(correspondance_matrix, cost_matrix, gamma):
    
    return np.sum(correspondance_matrix * np.log(correspondance_matrix)) + gamma * np.sum(corresponance_matrix * cost_matrix)

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('moscow.csv')

In [9]:
df.head()

,area_i,area_j,live_i_work_j,time_ij,dist_ij
0,1,1,40,26,2.2
1,1,2,4,49,8.1
2,1,3,3,35,6.8
3,1,4,2,75,9.3
4,1,5,2,30,8.0


In [10]:
size = df.area_i.max()

In [11]:
df.area_i = df.area_i.astype(int)
df.area_j = df.area_j.astype(int)

In [12]:
times_matrix, distance_matrix, corresponance_matrix = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))

In [13]:
for i in df.index:
    
    area_i, area_j, live_work_ij, time_ij, dist_ij = df.loc[i]
    area_i, area_j = int(area_i) - 1, int(area_j) - 1
    
    times_matrix[area_i, area_j] = time_ij
    distance_matrix[area_i, area_j] = dist_ij
    corresponance_matrix[area_i, area_j] = live_work_ij

In [14]:
L = 1
L_new = None
a = 0
# size = 10
beta = 1

# T = np.matrix([np.random.uniform(size=size) for _ in range(size)])
# T /= T.sum(axis=1)

T = 2 * np.matrix(times_matrix ** 4)
T /= T.sum(axis=1)

# l, w = np.random.uniform(0, 1, size), np.random.uniform(0, 1, size)  
l, w = df.groupby('area_i')['live_i_work_j'].sum(), df.groupby('area_j')['live_i_work_j'].sum()
l /= l.sum()
w /= w.sum()

lambdas_l, lambdas_w = np.zeros(size), np.zeros(size)
x, y, v = np.zeros(2 * size), np.zeros(2 * size), np.zeros(2 * size)

d_hat = 1

In [15]:
eps_f = 0.001

In [16]:
while True:
    if L_new is not None:
        L = L_new
        
    L_new = L / 2
    
    while True:
    
        a_new = 1 / (2 * L_new) + np.sqrt(1 / (4 * L_new ** 2) + a ** 2 * L / L_new)
        tau = 1 / (a_new * L_new)
        
        y = tau * v + (1 - tau) * x

        lambdas_l, lambdas_w = y[:size].copy(), y[size:].copy()
        
        B = np.exp(-beta * T - lambdas_l - lambdas_w)
        grad_phi_1, grad_phi_2, grad_phi_value = grad_phi(y[:size], y[size:], B)
        
        if np.linalg.norm(grad_phi_1) >= np.linalg.norm(grad_phi_2):
#             lambdas_l_new = lambdas_l + np.log(l) - np.log(np.exp((-beta * T.T - lambdas_l).T - lambdas_w).sum(axis=1).A1)
            lambdas_l_new = (lambdas_l + np.log(l) - np.log(np.dot(B, np.ones(size))).A1).values
            lambdas_w_new = lambdas_w.copy() 
        else:
            lambdas_l_new = lambdas_l.copy()
            lambdas_w_new = (lambdas_w + np.log(w) - np.log(np.dot(B.T, np.ones(size))).A1).values
#             lambdas_w_new = lambdas_w + np.log(w) - np.log(np.exp((-beta * T.T - lambdas_l).T - lambdas_w).sum(axis=0).A1)
        
        x_new = np.concatenate((lambdas_l_new, lambdas_w_new))
        v_new = v - a_new * grad_phi_value
        
        B_x_new = np.exp(-beta * T - x_new[:size] - x_new[size:])
        print(phi(x_new, l, w, B_x_new) - phi(y, l, w, B) - np.linalg.norm(grad_phi_value) ** 2 / (2 * L_new))

        if phi(x_new, l, w, B_x_new) <= phi(y, l, w, B) - np.linalg.norm(grad_phi_value) ** 2 / (2 * L_new):
            d = B / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
            d_hat_new = (a_new * d + L * a ** 2 * d_hat) / (L_new * a_new ** 2)
            break
        
        L_new *= 2
        print('L_new', L_new)
        
        
    print(abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B_x_new)))
    print(np.linalg.norm(np.dot(d_hat_new.T, np.ones(size).T).A1 - l), np.linalg.norm(np.dot(d_hat_new, np.ones(size).T).A1 - w))
    
    if abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B)) < eps_f:
        break
    
    d_hat = d_hat_new.copy()


<class 'numpy.ndarray'>
1.2920148928113506
L_new 1.0
<class 'numpy.ndarray'>
1.3374902631872028
L_new 2.0
<class 'numpy.ndarray'>
1.360227948375129
L_new 4.0
<class 'numpy.ndarray'>
1.3715967909690918
L_new 8.0
<class 'numpy.ndarray'>
1.3772812122660734
L_new 16.0
<class 'numpy.ndarray'>
1.3801234229145642
L_new 32.0
<class 'numpy.ndarray'>
1.3815445282388095
L_new 64.0
<class 'numpy.ndarray'>
1.3822550809009322
L_new 128.0
<class 'numpy.ndarray'>
1.3826103572319937
L_new 256.0
<class 'numpy.ndarray'>
1.3827879953975242
L_new 512.0
<class 'numpy.ndarray'>
1.3828768144802897
L_new 1024.0
<class 'numpy.ndarray'>
1.3829212240216722
L_new 2048.0
<class 'numpy.ndarray'>
1.3829434287923636
L_new 4096.0
<class 'numpy.ndarray'>
1.3829545311777092
L_new 8192.0
<class 'numpy.ndarray'>
1.382960082370382
L_new 16384.0
<class 'numpy.ndarray'>
1.3829628579667186
L_new 32768.0
<class 'numpy.ndarray'>
1.3829642457648867
L_new 65536.0
<class 'numpy.ndarray'>
1.3829649396639707
L_new 131072.0
<class 'nu

L_new 4.253529586511731e+37
<class 'numpy.ndarray'>
1.382965633563055
L_new 8.507059173023462e+37
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.7014118346046923e+38
<class 'numpy.ndarray'>
1.382965633563055
L_new 3.402823669209385e+38
<class 'numpy.ndarray'>
1.382965633563055
L_new 6.80564733841877e+38
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.361129467683754e+39
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.722258935367508e+39
<class 'numpy.ndarray'>
1.382965633563055
L_new 5.444517870735016e+39
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.0889035741470031e+40
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.1778071482940062e+40
<class 'numpy.ndarray'>
1.382965633563055
L_new 4.3556142965880123e+40
<class 'numpy.ndarray'>
1.382965633563055
L_new 8.711228593176025e+40
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.742245718635205e+41
<class 'numpy.ndarray'>
1.382965633563055
L_new 3.48449143727041e+41
<class 'numpy.ndarray'>
1.382965633563055
L_new 6.9689828745

L_new 7.237005577332262e+75
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.4474011154664524e+76
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.894802230932905e+76
<class 'numpy.ndarray'>
1.382965633563055
L_new 5.78960446186581e+76
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.157920892373162e+77
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.315841784746324e+77
<class 'numpy.ndarray'>
1.382965633563055
L_new 4.631683569492648e+77
<class 'numpy.ndarray'>
1.382965633563055
L_new 9.263367138985296e+77
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.8526734277970591e+78
<class 'numpy.ndarray'>
1.382965633563055
L_new 3.7053468555941183e+78
<class 'numpy.ndarray'>
1.382965633563055
L_new 7.410693711188237e+78
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.4821387422376473e+79
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.9642774844752946e+79
<class 'numpy.ndarray'>
1.382965633563055
L_new 5.928554968950589e+79
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.18571099

L_new 9.85050154909862e+114
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.970100309819724e+115
<class 'numpy.ndarray'>
1.382965633563055
L_new 3.940200619639448e+115
<class 'numpy.ndarray'>
1.382965633563055
L_new 7.880401239278896e+115
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.5760802478557792e+116
<class 'numpy.ndarray'>
1.382965633563055
L_new 3.1521604957115583e+116
<class 'numpy.ndarray'>
1.382965633563055
L_new 6.304320991423117e+116
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.2608641982846233e+117
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.5217283965692467e+117
<class 'numpy.ndarray'>
1.382965633563055
L_new 5.0434567931384933e+117
<class 'numpy.ndarray'>
1.382965633563055
L_new 1.0086913586276987e+118
<class 'numpy.ndarray'>
1.382965633563055
L_new 2.0173827172553973e+118
<class 'numpy.ndarray'>
1.382965633563055
L_new 4.034765434510795e+118
<class 'numpy.ndarray'>
1.382965633563055
L_new 8.06953086902159e+118
<class 'numpy.ndarray'>
1.382965633563055
L

OverflowError: (34, 'Result too large')

In [ ]:
np.dot(d_hat_new.T, np.ones(size)).A1 - l

In [ ]:
number_of_people = df.groupby('area_j')['live_i_work_j'].sum().sum()

In [ ]:
d_restored = d_hat_new * number_of_people

In [ ]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2

In [ ]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2